# Défi quotidien : comment affiner les LLM avec LoRA


Les méthodes de réglage fin efficace des paramètres (PEFT) , comme LoRA, répondent aux défis du réglage fin des grands modèles de langage (LLM) en ne mettant à jour qu'un petit sous-ensemble des paramètres du modèle. Cette approche réduit considérablement les coûts de calcul et de stockage, rendant le réglage fin des LLM plus accessible. Les techniques PEFT permettent aux développeurs d'adapter des modèles pré-entraînés à des tâches spécifiques sans avoir à réentraîner l'ensemble du modèle, ce qui accélère les cycles de développement et réduit la consommation de ressources.
Vous les mettrez en œuvre pour relever ce défi.



👩‍🏫 👩🏿‍🏫 Ce que vous apprendrez
Comment appliquer l’adaptation de bas rang (LoRA) à un modèle de langage pré-entraîné.
Comment affiner un modèle adapté à LoRA à l'aide de la bibliothèque PEFT Hugging Face.
Comment enregistrer et charger un modèle LoRA affiné.
Comment effectuer une inférence à l’aide d’un modèle LoRA affiné.


🛠️ Ce que vous allez créer
Un modèle de langage affiné qui génère du texte basé sur un ensemble de données spécifique de citations, en utilisant LoRA.


Ensemble de données
L'ensemble de données « Abirate/english_quotes », en particulier un échantillon de 10 % de la répartition de la formation.


Tâche
Installer les bibliothèques nécessaires (PEFT, jeux de données).
Chargez un modèle de langage pré-entraîné (bigscience/bloomz-560m) et son tokeniseur.
Chargez l’ensemble de données et prétraitez-le pour le modèle.
Configurez LoRA en utilisant LoraConfig.
Appliquez LoRA au modèle pré-entraîné à l'aide de get_peft_model.
Configurez les arguments de formation à l'aide de TrainingArguments.
Initialisez et entraînez le modèle à l’aide de Trainer.
Enregistrez le modèle LoRA affiné.
Chargez le modèle LoRA enregistré pour l'inférence à l'aide de PeftModel.from_pretrained.
Générer du texte à l’aide du modèle affiné et du tokenizer.

%pip install peft==0.4.0

mkdir cache

!pip install datasets

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = 
tokenizer = 
foundation_model = 

data =  # Sample 10%
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)
train_sample = data.select(range(5))
display(train_sample)

import peft
from peft import LoraConfig, get_peft_model

#Fill in `r=1` and `target_modules`.
lora_config = LoraConfig(
    r=,
    lora_alpha=, # a scaling factor that adjusts the magnitude of the weight matrix. Usually set to 1
    target_modules=,
    lora_dropout=,
    bias="none", # this specifies if the bias parameter should be trained.
    task_type="CAUSAL_LM"
)

#Add the adapter layers to the foundation model to be trained
peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.print_trainable_parameters())


### Fill out the `Trainer` class. 

import transformers
from transformers import TrainingArguments, Trainer
import os

output_directory = os.path.join("../cache/working", "peft_lab_outputs")
training_args = TrainingArguments(
    report_to="none",
    output_dir=output_directory,
    auto_find_batch_size=,
    learning_rate= 3e-2, # Higher learning rate than full fine-tuning.
    num_train_epochs=,
    use_cpu=True
)

trainer = Trainer(
    model=,
    args=,
    train_dataset=e,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

### Load the PEFT model using pre-defined LoRA configs and foundation model. We set `is_trainable=False` to avoid further training.

import time

time_now = 
peft_model_path = os.path.join(output_directory, f"peft_model_{time_now}")
trainer.model.save_pretrained(peft_model_path)

### Generate output tokens

inputs = tokenizer("Two things are infinite: ", return_tensors="pt")
outputs = peft_model.generate(
    ...
    )

print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

## Étape 1 : Installer les bibliothèques

In [ ]:
!pip install peft==0.4.0 datasets transformers accelerate


## Étape 2 : Charger le modèle et le tokenizer

In [2]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "bigscience/bloomz-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)


c:\Users\chume\.conda\envs\llama\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\chume\.conda\envs\llama\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\chume\.cache\huggingface\hub\models--bigscience--bloomz-560m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer

Avertissements mineurs, pas bloquants. Corrige si tu veux un environnement propre et plus efficace.

## Étape 3 : Charger et prétraiter les données

In [3]:
data = load_dataset("Abirate/english_quotes", split="train[:10%]")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)
train_sample = data.select(range(5))
print(train_sample)


c:\Users\chume\.conda\envs\llama\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\chume\.cache\huggingface\hub\datasets--Abirate--english_quotes. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map: 100%|██████████| 251/251 [00:00<00:00, 15686.77 examples/s]

Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 5
})


 **Analyse rapide :**

*  **Avertissement symlink toujours présent** : même cause que précédemment → Windows sans symlink actif → **pas bloquant** mais plus de stockage utilisé.

*  **Chargement Dataset réussi** :

   * **251 exemples** extraits (10 % du dataset),
   * **5 échantillons sélectionnés** pour l’entraînement,
    * Colonnes bien préparées : `input_ids`, `attention_mask` → **prétraitement OK**.

---

 **Conclusion** :
**Pas d'erreur critique**. Je peux continuer l'entraînement. Pour optimiser l’espace disque, je dois activer **Developer Mode**, sinon ignorer.


## Étape 4 : Configurer LoRA

In [4]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=1,
    lora_alpha=1,
    target_modules=["query_key_value"],  # typique pour Bloom
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.print_trainable_parameters())


trainable params: 98,304 || all params: 559,312,896 || trainable%: 0.01757585078102687
None


 **Analyse rapide** :

* **Paramètres entraînables** : 98 304
* **Paramètres totaux** : 559,3 millions
* **Proportion entraînée** : \~**0,0176 %**

 **Interprétation** :

* **Normal avec LoRA** : seul un minuscule sous-ensemble des poids (couches LoRA) est entraîné.
* **Avantages** :

  * **Moins de ressources**, entraînement rapide.
  * **Faible risque de surapprentissage** sur petit dataset.
* **Inconvénient** :

  * Pas adapté si tu veux modifier en profondeur les capacités du modèle → LoRA = **affinage léger et ciblé**.

 **Conclusion** : le résultat est attendu et confirme que **LoRA fonctionne comme prévu**.


## Étape 5 : Configurer TrainingArguments

In [5]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os

output_directory = "../cache/working/peft_lab_outputs"

training_args = TrainingArguments(
    report_to="none",
    output_dir=output_directory,
    auto_find_batch_size=True,
    learning_rate=3e-2,
    num_train_epochs=3,
    use_cpu=True
)


## Étape 6 : Entraîner le modèle

In [6]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_sample,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()


100%|██████████| 3/3 [00:04<00:00,  1.42s/it]

{'train_runtime': 4.2541, 'train_samples_per_second': 3.526, 'train_steps_per_second': 0.705, 'train_loss': 3.142648696899414, 'epoch': 3.0}


TrainOutput(global_step=3, training_loss=3.142648696899414, metrics={'train_runtime': 4.2541, 'train_samples_per_second': 3.526, 'train_steps_per_second': 0.705, 'total_flos': 1388070051840.0, 'train_loss': 3.142648696899414, 'epoch': 3.0})

 **Analyse rapide des résultats d'entraînement** :

* **Durée d'entraînement** : 4,25 secondes → **très rapide**, logique vu la petite taille de données et LoRA.
* **Vitesse** : \~3,5 échantillons/seconde, 0,7 étape/seconde → **efficace**, normal avec CPU.
* **Loss final** : 3,14 → **cohérent** pour un modèle pré-entraîné sur un jeu de données minuscule (5 exemples sur 3 époques).
* **Époques** : 3 → **conforme** au paramétrage.

 **Conclusion** : entraînement **ultra-rapide**, **loss stable** mais attention :

* **dataset trop petit** → perte peu significative.
* Utilisable uniquement pour **démonstration rapide**, pas production sérieuse.


## Étape 7 : Enregistrer le modèle

In [7]:
import time

time_now = int(time.time())
peft_model_path = os.path.join(output_directory, f"peft_model_{time_now}")
trainer.model.save_pretrained(peft_model_path)


## Étape 8 : Charger et inférence

In [8]:
from peft import PeftModel

peft_model = PeftModel.from_pretrained(foundation_model, peft_model_path, is_trainable=False)

inputs = tokenizer("Two things are infinite: ", return_tensors="pt")
outputs = peft_model.generate(input_ids=inputs["input_ids"], max_new_tokens=30)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


['Two things are infinite:  the universe and the universe.” And that is the universe and the universe.” And that is the universe and the universe.”” And that']


 **Analyse rapide du résultat généré** :

* **Problème clair** : **répétitions absurdes** → "the universe and the universe..." boucle incohérente.
* **Cause probable** :

  *  **Dataset minuscule** → 5 exemples = **surapprentissage massif**, absence de généralisation.
  *  **Pas de régulation du modèle** : pas de `temperature`, `top_k`, `top_p` → **sorties déterministes** qui accentuent la répétition.

---

 **Conclusion directe** :

* Avec **si peu de données**, le modèle **répète mécaniquement** ce qu'il a vu → résultat attendu.
*  **Solution rapide** : augmenter le nombre d'exemples, ajouter :

```python
outputs = peft_model.generate(
    input_ids=inputs["input_ids"],
    max_new_tokens=30,
    temperature=0.9,
    top_k=50,
    top_p=0.95
)
```

 Meilleure **diversité** dans la génération.


In [10]:
inputs = tokenizer("Two things are infinite: ", return_tensors="pt")

outputs = peft_model.generate(
    input_ids=inputs["input_ids"],
    max_new_tokens=30,          # Limite la longueur de la réponse
    temperature=0.9,            # Introduit de la diversité (plus élevé = plus créatif)
    top_k=50,                   # Ne garde que les 50 tokens les plus probables
    top_p=0.95,                 # Nucleus sampling : considère les tokens qui totalisent 95% de la probabilité
    do_sample=True              # Active l'échantillonnage aléatoire
)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


['Two things are infinite:  the universe and the universe.” If God is only like a book, what better book is sure to be in the universe than a book that is']


 **Analyse rapide du nouveau résultat** :

*  **Amélioration claire** : moins de répétitions mécaniques, une **phrase construite** avec une idée cohérente.
*  **Limite persistante** :

   * Le modèle reste **obsédé par "the universe"**, car le **dataset reste trop pauvre (5 exemples)**.
   * Mélange de **structure de citation** sans vrai fond de sens → typique d'un modèle **surentraîné sur très peu de données**.

---

 **Conclusion directe** :

*  Génération plus **variée** grâce aux paramètres (`temperature`, `top_k`, `top_p`).
*  **Problème fondamental** = **dataset insuffisant**.
*  **Solution optimale** : utiliser un **dataset plus large** (ex : tout le split `train[:50%]`), **nettoyer les données** et éventuellement augmenter les epochs **uniquement si le dataset est enrichi**.


## Bilan

 **Objectif atteint** : entraînement rapide d'un LLM (BloomZ-560m) avec **LoRA** sur un petit dataset spécifique (citations), sans réentraîner tout le modèle.

 **Méthode efficace** :

* **Peu de ressources** (seules quelques couches sont ajustées).
* **Temps d'entraînement réduit**.
* **Simplicité via PEFT (Hugging Face)**.

 **Points clés** :

* Utilisation d'un modèle pré-entraîné.
* Adaptation rapide avec LoRA (r=1).
* Validation pratique avec génération de texte.

 **Conclusion** : LoRA rend l'adaptation des LLM **rapide, légère et efficace**, parfait pour des cas d’usage spécifiques.
